In [ ]:
import glob
import numpy as np
import random

import cv2
import matplotlib.pyplot as plt
from PIL import Image

from torchvision import transforms as pil_transforms
from torchvision.transforms import functional as F_pil

import sys
sys.path.insert(0, '..')
from opencv_transforms import transforms
from opencv_transforms import functional as F

from setup_testing_directory import get_testing_directory

In [ ]:
datadir = get_testing_directory()
print(datadir)

In [ ]:
train_images = glob.glob(datadir + '/**/*.JPEG', recursive=True)
train_images.sort()
print('Number of training images: {:,}'.format(len(train_images)))

In [ ]:
random.seed(1)
imfile = random.choice(train_images)

In [ ]:
def plot_pil_and_opencv(pil_image, opencv_image, orientation='row'):
    if orientation == 'row':
        rows, cols = 1,3
        size = (8, 4)
    else: 
        rows, cols = 3,1
        size = (12, 6)
    fig, axes = plt.subplots(rows, cols,figsize=size)
    ax = axes[0]
    ax.imshow(pil_image)
    ax.set_title('PIL')

    ax = axes[1]
    ax.imshow(opencv_image)
    ax.set_title('opencv')

    ax = axes[2]
    l1 = np.abs(pil_image - opencv_image).mean(axis=2)
    ax.imshow(l1)
    ax.set_title('| PIL - opencv|\nMAE:{:.4f}'.format(l1.mean()))
    plt.tight_layout()
    plt.show()

In [ ]:
pil_image = Image.open(imfile)
image = cv2.cvtColor(cv2.imread(imfile, 1), cv2.COLOR_BGR2RGB)

In [ ]:
plot_pil_and_opencv(pil_image, image)

In [ ]:
pil_resized = pil_transforms.Resize((224, 224))(pil_image)

In [ ]:
resized = transforms.Resize(224)(image)

In [ ]:
plot_pil_and_opencv(pil_resized, resized)
plt.show()

In [ ]:
def L1(pil: Image, image: np.ndarray) -> float:
    return np.mean(np.abs(np.asarray(pil) - image))

In [ ]:
TOL = 1e-4

l1 = L1(pil_resized, resized)
assert l1 - 88.9559 < TOL

In [ ]:
random.seed(1)
pil = pil_transforms.RandomRotation(10)(pil_image)
random.seed(1)
np_img = transforms.RandomRotation(10)(image)

In [ ]:
plot_pil_and_opencv(pil, np_img)

In [ ]:
pil = pil_transforms.FiveCrop((224, 224))(pil_image)
cv  = transforms.FiveCrop((224,224))(image)

In [ ]:
pil_stacked = np.hstack([np.asarray(i) for i in pil])
cv_stacked = np.hstack(cv)

plot_pil_and_opencv(pil_stacked, cv_stacked, orientation='col')

In [ ]:
pil_stacked.shape

In [ ]:
l1